In [ ]:
cd ..

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
datapath = '/scr1/prgiven/microseismic-detection-ml/'
input_file_pattern = os.path.join(datapath, 'processed_windows/', '*.npy') # replace me - windowed files            
filenames = sorted(tf.io.gfile.glob(input_file_pattern))
print('Found {} files'.format(len(filenames)))

In [ ]:
job_id = 'train_20210611_025347_cnn2d_modular_010_' # Replace me
ckpt = '{}/models/{}/ckpt'.format(datapath, job_id)
model = tf.keras.models.load_model(ckpt)
model.compile()

In [ ]:
def run_prediction(filename, model):
    data = np.load(filename)
    inputs = preprocess(inputs)
    return model.predict(inputs, use_multiprocessing=True)

def preprocess(data):
    # this should be the same preprocessing as when you converted TFRecords
    # crop it to input_height and width (cropping at the center)
    data = data.reshape((1, 288, 512, 1)) # adjust numbers
    data = np.clip(data, -0.1, 0.1) / 0.03 # stats
    return np.float32(data)

logits = np.zeros(len(filenames))
for i, filename in enumerate(filenames):
    output = run_prediction(filename, model)
    logits[i] = output

In [ ]:
probability = tf.nn.sigmoid(logits).numpy()